In [ ]:
!pip install trl sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.4/517.4 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.0 MB/s eta 0:00:00


In [ ]:
import os, json, random
from pathlib import Path
from typing import Dict, Any, List

import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, PeftModel
import trl
from trl import SFTTrainer, SFTConfig
import sacrebleu

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

os.environ.setdefault("PYTORCH_ENABLE_MPS_FALLBACK", "1")
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print("device:", device)
print("torch:", torch.__version__)
print("trl:", trl.__version__)


device: cuda
torch: 2.9.0+cu126
trl: 0.26.1


In [ ]:
import json
from pathlib import Path
from datasets import Dataset

DATA_PATH = Path("train_data.json")

# 1. 데이터 로드 (JSONL 형식 또는 JSON List 형식 모두 대응)
text = DATA_PATH.read_text(encoding="utf-8").strip()

if not text:
    raise RuntimeError("train_data.json이 비어 있습니다.")

raw_data = []

try:
    # 한 줄씩 읽기 (JSONL) 시도
    for line in text.splitlines():
        if line.strip():
            raw_data.append(json.loads(line))
except json.JSONDecodeError:
    raw_data = json.loads(text)

# 2. 데이터 정제 함수
def normalize_item(item: dict) -> dict:
    # idiom 딕셔너리에서 term과 meaning을 추출하여 상위 레벨로 올림
    idiom_data = item.get("idiom", {})

    return {
        "instruction": item["instruction"].strip(),
        "input": item["input"].strip(),
        "output": item["output"].strip(),
        "term": idiom_data.get("term", "").strip(),
        "meaning": idiom_data.get("meaning", "").strip()
    }

# 3. 데이터셋 생성
norm = [normalize_item(x) for x in raw_data]
ds = Dataset.from_list(norm).train_test_split(test_size=0.1, seed=SEED)

print(f"✅ 데이터: train={len(ds['train'])}, dev={len(ds['test'])}")

print("예시 데이터 항목:")
ds['train'][1]

✅ 데이터: train=540, dev=60
예시 데이터 항목:


{'instruction': "Don't translate it in Korean, but translate it according to Korean culture",
 'input': 'So far so good with the new diet.',
 'output': '다이어트 아직까진 잘 버티고 있어.',
 'term': 'So far so good',
 'meaning': '아직까지는 좋다'}

In [ ]:
SYSTEM_PROMPT = """
You are a professional translator who specializes in translating English slang and memes into Korean internet slang and trendy expressions.
Your goal is to make the translation sound like a "close friend" or a "Korean netizen" speaking.

**CRITICAL RULES:**
1. **Never use polite language (Honorifics/Jon-dae-mal).** Use ONLY casual speech (Banmal).
2. Do not translate literally. Use Korean slang, memes, and community vibes aggressively.
3. If the original text is sarcastic or rude, preserve that tone perfectly.
4. Output ONLY the Korean translation. No explanations.
"""



def build_prompt(example: Dict[str, Any]) -> str:
    ref = ""
    if example.get("term") or example.get("meaning"):
        ref = f"[Note]\n- Expression: {example.get('term','')}\n- Meaning: {example.get('meaning','')}\n\n"
    user = (
        f"지시사항: {example['instruction']}\n\n"
        f"{ref}"
        f"입력 문장: {example['input']}\n"
        f"출력 조건: 한국어 번역 1개만 출력"
    )
    return (
        "### System:\n"
        f"{SYSTEM_PROMPT.strip()}\n\n"
        "### User:\n"
        f"{user}\n\n"
        "### Assistant:\n"
    )

def to_prompt_completion(ex: Dict[str, Any]) -> Dict[str, str]:
    return {"prompt": build_prompt(ex), "completion": ex["output"]}

pc = ds.map(to_prompt_completion, remove_columns=ds["train"].column_names)
print(pc["train"][0]["prompt"][:600])
print("completion:", pc["train"][0]["completion"])


Map:   0%|          | 0/540 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

### System:
You are a professional translator who specializes in translating English slang and memes into Korean internet slang and trendy expressions.
Your goal is to make the translation sound like a "close friend" or a "Korean netizen" speaking.

**CRITICAL RULES:**
1. **Never use polite language (Honorifics/Jon-dae-mal).** Use ONLY casual speech (Banmal).
2. Do not translate literally. Use Korean slang, memes, and community vibes aggressively.
3. If the original text is sarcastic or rude, preserve that tone perfectly.
4. Output ONLY the Korean translation. No explanations.

### User:
지시사항:
completion: 이 버그 고친 걔는 진짜 빛이고 소금(갓)이야.


In [ ]:
MODEL_ID = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"
OUTPUT_DIR = "./V3_hyperclova-translator-mps"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer.padding_side = "right"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float32,  # MPS 안정성
    trust_remote_code=True
)
base_model.config.use_cache = False
print("✅ 모델 로드 완료")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

✅ 모델 로드 완료


In [ ]:
preferred_targets = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

def guess_target_modules(model) -> List[str]:
    names = set()
    for n,_ in model.named_modules():
        names.add(n.split(".")[-1])
    return [t for t in preferred_targets if t in names]

targets = guess_target_modules(base_model)
if not targets:
    targets = "all-linear"
    print("⚠️ target_modules 자동 탐지 실패 → all-linear 사용")
else:
    print("✅ target_modules:", targets)

peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=targets,
    bias="none",
    task_type="CAUSAL_LM",
)

training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=200,
    bf16=False,
    fp16=False,
    optim ="adamw_torch",
    report_to="none",
    packing=False,
    seed=SEED,
    completion_only_loss=True,  # ✅ 핵심
)

trainer = SFTTrainer(
    model=base_model,
    args=training_args,
    train_dataset=pc["train"],
    eval_dataset=pc["test"],
    peft_config=peft_config,
)

print("✅ trainer 준비 완료")


✅ target_modules: ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/540 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/540 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/540 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/60 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/60 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/60 [00:00<?, ? examples/s]

✅ trainer 준비 완료


In [ ]:
print("🚀 학습 시작")
trainer.train()
print("✅ 학습 종료")

trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"🎉 저장 완료: {OUTPUT_DIR}")


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 100275}.


🚀 학습 시작


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
50,2.176600,2.042578,2.955153,90578.000000,0.554477
100,1.570400,2.038223,2.537286,180345.000000,0.568551
150,0.946300,2.158222,2.260243,270217.000000,0.564391
200,0.894900,2.133584,2.364092,360749.000000,0.553312
250,0.450200,2.492612,2.005729,450420.000000,0.554767
300,0.157000,2.633188,1.914964,540259.000000,0.540290
350,0.068500,2.860275,1.824161,630113.000000,0.542143
400,0.073200,2.885980,1.838016,720796.000000,0.528133
450,0.020800,2.983797,1.787246,810429.000000,0.532293
500,0.011600,3.080172,1.732358,900488.000000,0.531627


✅ 학습 종료
🎉 저장 완료: ./V3_hyperclova-translator-mps


In [ ]:
import shutil
import os

source = '/content/V3_hyperclova-translator-mps' # 옮길 폴더 이름
destination = '/content/drive/MyDrive/my_lab_folder' # 드라이브 내 저장할 경로

# 폴더 이동 실행
try:
    shutil.move(source, destination)
    print(f"이동 완료: {destination}")
except Exception as e:
    print(f"오류 발생: {e}")

이동 완료: /content/drive/MyDrive/my_lab_folder
